# IFT6758 Data Science Kaggle competition


Team - Four Geese
>
Name | Student ID | Kaggle Username
--- | --- | ---
François Doan-Pope | 20054932 | franckdp
Sina Sarparast | 20181256 | Sina1374
Joshua Amelia | 20146389 | joshuaamelia
Carryll Yu Ping Liang | 20051432 | yu4kaggle
>

# Grabbing data from Kaggle

You can skip this section and download the dataset manually.

This section is for google colab only. First install kaggle api. Then from the profile setting activate the data api and download your Kaggle key (.json file).

update it as well

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 6.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=de99bcdd3d81454fdc5d0930e12e971923ac7413d9485b42b0b0211a3e0eb3a0
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Upload the key here

In [ ]:
from google.colab import files

files.upload()

Saving kaggle-3.json to kaggle-3.json


{'kaggle-3.json': b'{"username":"sina1374","key":"fc9770d07c6c9c1d817ce2340a15a5cc"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

download dataset using the key and the following command

In [ ]:
!kaggle competitions download -c ift6758-a20

ift6758-a20.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip ift6758-a20.zip

# Access data from google drive

This section has nothing to do with the competition. I'm just trying to make things easier for everyone.

You may need to install pydrive first.

In [ ]:
! pip install pydrive

We try to access the data set as a shared drive file. Here are the Drive file ids:

In [ ]:
train_csv_id = '13C3PPMOX8ZmAqUYVTugknL72yk6sblA8'
test_csv_id = '1rKugQNSo_6iILOsriGbS90fOCV5bfh9l'

We authenticate and then create a file reader

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download the files to the local colab storage.
You need to do this each time you open the colab file, since google colab doesn't store files between sessions. Refresh the files screen to see downloaded files.

In [ ]:
train_csv = drive.CreateFile({'id': train_csv_id})
train_csv.GetContentFile('train.csv', mimetype='csv')
test_csv = drive.CreateFile({'id': test_csv_id})
test_csv.GetContentFile('test.csv', mimetype='csv')
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))

# Import libraries and data & seed

In [ ]:
import pandas as pd
import numpy as np
import time
from numpy.random import seed
import tensorflow as tf

seed(12)
tf.random.set_seed(24)

%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

# Model retained for Kaggle and its preprocessing

read the CSV files to pandas dataframes

In [ ]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

## Preprocessing on user's locations

### Functions to extract locations (read note)

Note: Without API key, this will not work. Skip to "Loading back our extracted data"

In [ ]:
from geopy.geocoders import GoogleV3
from google.colab import drive
drive.mount('drive')

path = 'define your drive path'

train_location = train_set[['Id', 'Location', 'User Time Zone']]
test_location = test_set[['Id', 'Location', 'User Time Zone']]

geolocator = GoogleV3(api_key='ENTER YOUR API KEY)

In [ ]:
def extract_country(address):
  split = address.rsplit(',', 1)
  
  if len(split) != 1:
    country = split[1].strip()
  else:
    country = split[0]

  return country

def find_country(array):

  country_list = []

  for value in array:
    if pd.notna(value):
      loc = geolocator.geocode(value, language='en')
      if loc:
        country = extract_country(loc.address)
        country_list.append(country)
      else:
        country_list.append(None)
    else:
      country_list.append(None)

  return country_list

Extracting location from location column.

In [ ]:
# Training set

train_location_ext = find_country(train_location.iloc[:,1].values)
df_temp = pd.DataFrame(train_location_ext)
df_location = pd.concat([train_set.iloc[:,0], df_temp], axis=1)
df_location.to_csv(path+'/train_location.csv')

# Testing set

df_temp_test = pd.DataFrame(test_location_ext)
df_location_test = pd.concat([test_set.iloc[:,0], df_temp_test], axis=1)
df_location_test.to_csv(path+'/test_location.csv')

Extraction location from time zone column

In [ ]:
# Training set

df_location2 = pd.merge(df_location, train_location, left_on='Id', right_on='Id', how='left')
tmz_location = df_location2[df_location2[0].isnull()]
train_tmz_location = find_country(tmz_location.iloc[:,3].values)
df_temp_tmz = pd.DataFrame(train_tmz_location, columns=['TMZ Location'])
df_location_tmz = pd.concat([tmz_location.reset_index(), df_temp_tmz.reset_index()], axis=1)

df_location_tmz.to_csv(path+'/train_location_tmz.csv')

# Testing set

df_location_test2 = pd.merge(df_location_test, test_location, left_on='Id', right_on='Id', how='left')
tmz_location_test = df_location_test2[df_location_test2[0].isnull()]
test_tmz_location = find_country(tmz_location_test.iloc[:,3].values)
df_temp_tmz_test = pd.DataFrame(test_tmz_location, columns=['TMZ Location'])
df_location_tmz_test = pd.concat([tmz_location_test.reset_index(), df_temp_tmz_test.reset_index()], axis=1)

df_location_tmz_test.to_csv(path+'/test_location_tmz.csv')

### Loading back our extracted data

In [ ]:
tmz_fields = ['Id', 'TMZ Location']
loc_names = ['Id', 'Location']

train_loc = pd.read_csv('https://raw.githubusercontent.com/francoisdoanp/projects/master/DS/train_location.csv', usecols=['Id', '0']).set_index('Id')
train_tmz = pd.read_csv('https://raw.githubusercontent.com/francoisdoanp/projects/master/DS/train_location_tmz.csv', usecols=tmz_fields).set_index('Id')
test_loc = pd.read_csv('https://raw.githubusercontent.com/francoisdoanp/projects/master/DS/test_location.csv').drop('Unnamed: 0', 1).set_index('Id')
test_tmz = pd.read_csv('https://raw.githubusercontent.com/francoisdoanp/projects/master/DS/test_location_tmz.csv', usecols=tmz_fields).set_index('Id')

### Transformations on location data

In [ ]:
tmpt = train_loc['0']
tmpt.update(train_tmz['TMZ Location'])
train_loc['0'] = tmpt
train_loc.reset_index(inplace=True)

tmpte = test_loc['0']
tmpte.update(test_tmz['TMZ Location'])
test_loc['0'] = tmpte
test_loc.reset_index(inplace=True)

# Manually replacing erroneous values

train_setna = ['International Date Line', '02000', '127051', '3 Chome−中２−１', '1591-9 경기도 안양시 동안구','125009', '603000','서울특별시 강남구 역삼동 719-5', '109012']
test_setna = ['번지 102호 1041-5 조은빌딩', '117105', '6-chōme−27−番３０号 新宿イーストサイドスクエア', '1−７９ アリオ川口1階', '690002', '02000']
loc_replace = {'Dubai - United Arab Emirates': 'United Arab Emirates', 'UK':'United Kingdom', 'Jeddah Saudi Arabia': 'Saudi Arabia', 'United States': 'USA', 'Al Qassim Saudi Arabia': 'Saudi Arabia', 
                 'Sheikh Mohammed bin Rashid Blvd - Dubai - United Arab Emirates': 'United Arab Emirates', 
                 'Sheikh Zayed Rd Near Financial Metro Station - Trade CentreTrade Centre 1 - Dubai - United Arab Emirates': 'United Arab Emirates', 'Singapore 207542': 'Singapore', 
                 'Sharjah - United Arab Emirates': 'United Arab Emirates', 'Al Sufouh 1 - Dubai - United Arab Emirates': 'United Arab Emirates', 
                 'Yas Island - Abu Dhabi - United Arab Emirates': 'United Arab Emirates', 'Al SufouhDubai Media City - Dubai - United Arab Emirates': 'United Arab Emirates', 'Singapore 039593': 'Singapore',
                 'Abu Dhabi - United Arab Emirates': 'United Arab Emirates', '98000 Monaco': 'Monaco', 'Persian Gulf (also known as the Arabian Gulf)': 'Persian Gulf', '4-chōme−4−番２号 東山ビルディング 5階': np.NaN,
               'The Bahamas': 'Bahamas', 'U.S. Virgin Islands': 'United States Virgin Islands'}

train_loc['0'].replace(train_setna, np.NaN, inplace=True)
test_loc['0'].replace(test_setna, np.NaN, inplace=True)
train_loc.replace({'0': loc_replace}, inplace=True)
test_loc.replace({'0': loc_replace}, inplace=True)

# Replacing nan's with unknown

train_loc.fillna('Unknown', inplace=True)
test_loc.fillna('Unknown', inplace=True)

train_loc.columns = ['Id', 'Location']
test_loc.columns = ['Id', 'Location']

# Replacing countries that are not in the test set as unknown
known_locations = train_loc['Location'].unique()
test_loc[~test_loc.Location.isin(known_locations)] = 'Unknown'

In [ ]:
train_set['Location'] = train_loc['Location']
test_set['Location'] = test_loc['Location']

## Preprocessing on color features

In [ ]:
from PIL import ImageColor

# Creating a dataframe for each features containing color values
train_texcolor = train_set['Profile Text Color']
train_pagcolor = train_set['Profile Page Color']
train_thmcolor = train_set['Profile Theme Color']

test_texcolor = test_set['Profile Text Color']
test_pagcolor = test_set['Profile Page Color']
test_thmcolor = test_set['Profile Theme Color']

In [ ]:
# Functions to extract colors from HEX values
def get_color(color):
  rgb = ImageColor.getcolor('#'+ str(color), "RGB")

  return [x/255.0 for x in rgb]

def get_all_colors(data):

  color_array = []

  for color in data:
    temp = get_color(color)
    color_array.append(temp)

  return np.array(color_array)

In [ ]:
# Clustering colors
from sklearn.cluster import KMeans

def cluster_colors(array, k):
  kmeans = KMeans(k)
  kmeans.fit(array)
  clustered_array = kmeans.predict(array)

  return kmeans, clustered_array

In [ ]:
from sklearn_pandas import CategoricalImputer

# Imputing values (which is the same as SimpleImputer based on version 2.0.0 update of sk_learnpandas)
imputer_cat = CategoricalImputer()
train_texcolor = imputer_cat.fit_transform(train_texcolor)
train_pagcolor = imputer_cat.fit_transform(train_pagcolor)
train_thmcolor = imputer_cat.fit_transform(train_thmcolor)

train_texcolor = get_all_colors(train_texcolor)
train_pagcolor = get_all_colors(train_pagcolor)
train_thmcolor = get_all_colors(train_thmcolor)

test_texcolor = imputer_cat.fit_transform(test_texcolor)
test_pagcolor = imputer_cat.fit_transform(test_pagcolor)
test_thmcolor = imputer_cat.fit_transform(test_thmcolor)

test_texcolor = get_all_colors(test_texcolor)
test_pagcolor = get_all_colors(test_pagcolor)
test_thmcolor = get_all_colors(test_thmcolor)

In [ ]:
kmeans_tex, train_tcolor = cluster_colors(train_texcolor, 12)
kmeans_pag, train_pcolor = cluster_colors(train_pagcolor, 12)
kmeans_thm, train_thcolor = cluster_colors(train_thmcolor, 12)

test_tcolor = pd.DataFrame(kmeans_tex.predict(test_texcolor), columns=['text_color'])
test_pcolor =  pd.DataFrame(kmeans_pag.predict(test_pagcolor), columns=['page_color'])
test_thcolor =  pd.DataFrame(kmeans_thm.predict(test_thmcolor), columns=['theme_color'])

train_tcolor = pd.DataFrame(train_tcolor, columns=['text_color'])
train_pcolor = pd.DataFrame(train_pcolor, columns=['page_color'])
train_thcolor = pd.DataFrame(train_thcolor, columns=['theme_color'])

In [ ]:
train_set.drop(columns=['Profile Text Color', 'Profile Page Color', 'Profile Theme Color'], inplace=True)
test_set.drop(columns=['Profile Text Color', 'Profile Page Color', 'Profile Theme Color'], inplace=True)

train_set = pd.concat([train_set, train_tcolor, train_pcolor, train_thcolor], axis=1)
test_set = pd.concat([test_set, test_tcolor, test_pcolor, test_thcolor], axis=1)

## Preprocessing functions

In [ ]:
import time
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler

# Function to impute missing values using IterativeImputer (more precisely, Bayesian Ridge)
def imputeMissing(df):
    cols =[
        'Profile Cover Image Status',
    ]
    df[cols]=df[cols].fillna('missing')

    cols =[
       'Num of Followers',
        'Num of People Following',
        'Num of Status Updates',
        'Num of Direct Messages',
        'Avg Daily Profile Visit Duration in seconds',
        'Avg Daily Profile Clicks',
    ]
    imp = IterativeImputer(missing_values=np.nan, initial_strategy='median', imputation_order='random', add_indicator=True)
    imp.fit(df[cols])

    df_tmp = imp.transform(df[cols])
    return df_tmp

# Function to modify some peculiar values
def cleanData(df):
    df['Location Public Visibility'] = df['Location Public Visibility'].str.lower()
    indicies = df['Location Public Visibility']=='??'
    df.loc[indicies,'Location Public Visibility'] = df['Location Public Visibility'].mode().item()
    indicies = df['Profile Category']==' ' 
    df.loc[indicies,'Profile Category'] = 'unknown'
    return df

# Functions for some feature engineering
# Length of username, boolean indicator for presence of personal URL and year of profile creation
def transformData(df):
    df['len User Name'] = df['User Name'].str.len()
    index = df['Personal URL'].isnull()
    df['Has Personal URL'] = 0
    df.loc[index,'Has Personal URL'] = 1
    df['Year Of Profile Creation'] = pd.DatetimeIndex(df['Profile Creation Timestamp']).year

    return df

# Transformations on some numerical features to have a more Gaussian-like distribution.
def getPowerTransformer(train_set):
    to_scale = ['Num of Followers',
                'Num of People Following',
                'Num of Status Updates',
                'Num of Direct Messages',
                'Total Daily Profile Visit Duration in seconds',
                'Total Clicks',
                'Avg Followers Per Year'
        ]

    pt = PowerTransformer()
    pt.fit(train_set[to_scale]) 
    return pt

def getMinMaxScaler(train_set):
    to_scale = [
                'Avg Daily Profile Visit Duration in seconds',
                'Avg Daily Profile Clicks',
                'len User Name'
            ]

    scaler = MinMaxScaler()
    scaler.fit(train_set[to_scale])
    return scaler

# One-hot encoding for three labels
def getLabelEncoders(train_set):
    cols = [
            'Profile Cover Image Status',
            'Is Profile View Size Customized?',
            'Location Public Visibility']
    labelEncoders = {}
    for col in cols:
        labelEncoders[col] = LabelEncoder()
        labelEncoders[col].fit( train_set[col] )
    return labelEncoders

# Putting it all together
def encodeAndScale(df, labelEncoders, powerTransformer, minMaxScaler):
    cols = [
            'User Language',
        'Profile Category',
        'Profile Verification Status',
        'Year Of Profile Creation',
        'theme_color',
        'text_color',
        'page_color',
        'Location'
    ]
    df = pd.get_dummies(df, columns=cols)

    cols = [
            'Profile Cover Image Status',
            'Is Profile View Size Customized?',
            'Location Public Visibility']
    for col in cols:
        df[col] = labelEncoders[col].transform( df[col] )

    to_remove = ['Id',
                'User Name',
                'Personal URL',
                'Profile Image',
                'UTC Offset',
                'Profile Creation Timestamp',
                ] 

    df = df.drop(columns=to_remove)

    to_scale = ['Num of Followers',
                'Num of People Following',
                'Num of Status Updates',
                'Num of Direct Messages',
                'Total Daily Profile Visit Duration in seconds',
                'Total Clicks',
                'Avg Followers Per Year'
        ]
    df[to_scale] = powerTransformer.transform(df[to_scale])


    to_scale = [
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks',
                    'len User Name'
                ]
    df[to_scale] = minMaxScaler.transform(df[to_scale])
    return df

## Applying the preprocessing on our training and testing data

In [ ]:
# Dropping location and user time zone and adding the country extracted for all observations

train_set.drop(columns=['Location', 'User Time Zone'], axis=1, inplace=True)
test_set.drop(columns=['Location', 'User Time Zone'], axis=1, inplace=True)

train_set = pd.concat([train_set, train_loc], axis=1)
test_set = pd.concat([test_set, test_loc], axis=1)

In [ ]:
# Applying our preprocessing functions

trainDf = train_set.copy()
testDf = test_set.copy()

train_tmp = imputeMissing(trainDf)
trainDf['Num of Followers'] = train_tmp[:,0]
trainDf['Num of People Following'] = train_tmp[:,1]
trainDf['Num of Status Updates'] = train_tmp[:,2]
trainDf['Num of Direct Messages'] = train_tmp[:,3]
trainDf['Avg Daily Profile Visit Duration in seconds'] = train_tmp[:,4]
trainDf['Avg Daily Profile Clicks'] = train_tmp[:,5]
trainDf['indc1'] = train_tmp[:,6]
trainDf['indc2'] = train_tmp[:,7]

# Feature engineering: Adding three variables from calculations based on available features.
trainDf['Total Daily Profile Visit Duration in seconds'] = trainDf['Avg Daily Profile Visit Duration in seconds'] * trainDf['Avg Daily Profile Clicks']
trainDf['Total Clicks'] = trainDf['Avg Daily Profile Clicks'] * 365 * (2020-pd.DatetimeIndex(trainDf['Profile Creation Timestamp']).year)
trainDf['Avg Followers Per Year'] = trainDf['Num of Followers']/(2020-pd.DatetimeIndex(trainDf['Profile Creation Timestamp']).year)

trainDf = cleanData(trainDf)
trainDf = transformData(trainDf)

les = getLabelEncoders(trainDf)
pt = getPowerTransformer(trainDf)
mms = getMinMaxScaler(trainDf)
trainDf = encodeAndScale(trainDf, les, pt, mms)
x_train = trainDf.drop(columns=["Num of Profile Likes"])
y_train = trainDf["Num of Profile Likes"]

test_tmp = imputeMissing(testDf)

testDf['Num of Followers'] = test_tmp[:,0]
testDf['Num of People Following'] = test_tmp[:,1]
testDf['Num of Status Updates'] = test_tmp[:,2]
testDf['Num of Direct Messages'] = test_tmp[:,3]
testDf['Avg Daily Profile Visit Duration in seconds'] = test_tmp[:,4]
testDf['Avg Daily Profile Clicks'] = test_tmp[:,5]
testDf['indc1'] = test_tmp[:,6]
testDf['indc2'] = test_tmp[:,7]

# Feature engineering: Adding three variables from calculations based on available features.
testDf['Total Daily Profile Visit Duration in seconds'] = testDf['Avg Daily Profile Visit Duration in seconds'] * testDf['Avg Daily Profile Clicks']
testDf['Total Clicks'] = testDf['Avg Daily Profile Clicks'] * 365 * (2020-pd.DatetimeIndex(testDf['Profile Creation Timestamp']).year)
testDf['Avg Followers Per Year'] = testDf['Num of Followers']/(2020-pd.DatetimeIndex(testDf['Profile Creation Timestamp']).year)

testDf = cleanData(testDf)
testDf = transformData(testDf)
x_test = encodeAndScale(testDf, les, pt, mms)
missingCols = set(x_train.columns) - set(x_test.columns)
for c in missingCols:
    x_test[c] = 0
x_test = x_test[x_train.columns]

## Loading the images from the competition



In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Function to load images from the file name in our dataframe
# Input: - Dataframe containing the filenames in column Profile Image
#        - The path where images are located.
# Output: Numpy array 
# Note: the images loaded from the Dataframe needs to be in the same order as the features from the training/testing set.

def load_images(dataframe, imagesPath):

  inputImages = []

  for fileName in dataframe.values:
    image = load_img(imagesPath + fileName[0])
    inputImages.append(img_to_array(image))

  return np.array(inputImages)

In [ ]:
# Loading dataframe with images filename

train_images = pd.read_csv('train.csv', usecols=['Profile Image'])
test_images =  pd.read_csv('test.csv', usecols=['Profile Image'])

In [ ]:
# Define the image's paths
path_train = './train_profile_images/profile_images_train/'
path_test = './test_profile_images/profile_images_test/'

# Import images based on DataFrame's Profile Image feature
training_imgs = load_images(train_images, path_train)
print('Training images shape: {}'.format(training_imgs.shape))
testing_imgs = load_images(test_images, path_test)
print('Training images shape: {}'.format(testing_imgs.shape))

# Normalizing pixels
training_imgs, testing_imgs = (training_imgs-127.5)/127.5, (testing_imgs-127.5)/127.5

Training images shape: (7500, 32, 32, 3)
Training images shape: (2500, 32, 32, 3)


In [ ]:
from sklearn.model_selection import train_test_split

trainX, valX, trainImg, valImg, trainY, valY = train_test_split(x_train, training_imgs,
                                                                y_train, test_size=0.20, random_state=42)

print("x_train:", x_train.shape)
print("training_imgs:", training_imgs.shape)
print("y_train:", y_train.shape)
print("="*50)
print("trainX:", trainX.shape)
print("valX:", valX.shape)
print("trainImg:", trainImg.shape)
print("valImg:", valImg.shape)
print("trainY:", trainY.shape)
print("valY:", valY.shape)
print("="*50)
print("x_test:", x_test.shape)
print("testing_imgs",testing_imgs.shape)

x_train: (7500, 236)
training_imgs: (7500, 32, 32, 3)
y_train: (7500,)
trainX: (6000, 236)
valX: (1500, 236)
trainImg: (6000, 32, 32, 3)
valImg: (1500, 32, 32, 3)
trainY: (6000,)
valY: (1500,)
x_test: (2500, 236)
testing_imgs (2500, 32, 32, 3)


## Neural Network model

### Image Data Generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Applying some image transformations
datagen = ImageDataGenerator(
          featurewise_center=False,  
          samplewise_center=True,  
          featurewise_std_normalization=False,   
          samplewise_std_normalization=False,  
          zca_whitening=False,  
          rotation_range=5,
          width_shift_range=0.1,
          height_shift_range=0.1, 
          horizontal_flip=True,  
          vertical_flip=False,
          brightness_range=[0.1,1.0],
          zoom_range=[0.2,1.0],
          rescale=1.0/255.0)  

### CNN + MLP

In [ ]:
from sklearn.model_selection import train_test_split

trainX, valX, trainImg, valImg, trainY, valY = train_test_split(x_train, training_imgs, y_train, test_size=0.3, random_state=42)

generator_train = datagen.flow([trainImg, trainX], y=trainY, shuffle=True)
generator_val = datagen.flow([valImg, valX], valY, shuffle=True)
generator_test = datagen.flow([testing_imgs, x_test], shuffle=False)

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate, Input
import tensorflow as tf

# Define inputs shape
num_features = x_train.shape[1]
size_imgs = training_imgs.shape[1:]

image_input = Input((size_imgs))
feature_input = Input((num_features,))

# Define CNN architecture - adjust hyperparameters as needed

conv_layer = Conv2D(32, (3,3), activation='relu')(image_input)
max_pool_layer = MaxPooling2D((2,2))(conv_layer)
conv_layer2 = Conv2D(16, (3, 3), activation='relu')(max_pool_layer)
max_pool_layer2 = MaxPooling2D((2,2))(conv_layer2)
flat_layer = Flatten()(max_pool_layer2)
dense_layer_cnn = Dense(8, activation='relu')(flat_layer)

# Define MLP Architecture - adjust hyperparameters as needed

dense_layer2 = Dense(128, activation='relu')(feature_input)
dropout2 = Dropout(0.5)(dense_layer2)
dense_layer3 = Dense(64, activation='relu')(feature_input)
dropout3 = Dropout(0.5)(dense_layer3)
dense_layer4 = Dense(32, activation='relu')(dropout3)

# Concatenate features (or MLP) and CNN - adjust hyperparameters as needed

concatenate_layer = Concatenate()([dense_layer_cnn, dense_layer4])
dense_layer_l3 = Dense(64, activation='relu')(concatenate_layer)
dropout_l = Dropout(0.5)(dense_layer_l3)
dense_layer_l4 = Dense(32, activation='relu')(dropout_l)
dropout_l2 = Dropout(0.5)(dense_layer_l4)
dense_layer_l5 = Dense(16, activation='relu')(dropout_l2)
dropout_l3 = Dropout(0.5)(dense_layer_l5)
dense_layer_l6 = Dense(8, activation='relu')(dropout_l3)
dropout_l4 = Dropout(0.5)(dense_layer_l6)
output = Dense(1, activation='relu')(dropout_l4)

model_cnnmlp = Model(inputs=[image_input, feature_input], outputs=output)

model_cnnmlp.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 30, 30, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 15, 15, 32)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 13, 13, 16)   4624        max_pooling2d_2[0][0]            
_______________________________________________________________________________________

In [ ]:
from keras.utils import plot_model

# Plot the model architecture
plot_model(model_cnnmlp)

In [ ]:
# Defining our RMSLE loss function

def RMSLE(y_pred, y_true): 
  y_pred= tf.cast(y_pred, tf.float64) 
  y_true= tf.cast(y_true, tf.float64) 
  y_pred=tf.nn.relu(y_pred) 
  return tf.sqrt(tf.reduce_mean(tf.math.squared_difference(tf.math.log1p(y_pred), tf.math.log1p(y_true))))

In [ ]:
# Fiting the model - adjust hyperparameters as needed.
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
rlr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=10)
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', mode='min')

model_cnnmlp.compile(optimizer='RMSProp', loss=RMSLE, metrics=['mean_squared_logarithmic_error'])
model_cnnmlp.fit(x=generator_train, validation_data=generator_val, epochs=200, callbacks=[callback, checkpoint, rlr_plateau])

Epoch 1/100
188/188 [==============================] - 1s 5ms/step - loss: 13.6178 - RMSLE: 3.5378 - val_loss: 6.8976 - val_RMSLE: 2.5994
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 9.0194 - RMSLE: 2.9701 - val_loss: 6.9372 - val_RMSLE: 2.6057
Epoch 3/100
188/188 [==============================] - 1s 4ms/step - loss: 8.4574 - RMSLE: 2.8788 - val_loss: 6.8962 - val_RMSLE: 2.5992
Epoch 4/100
188/188 [==============================] - 1s 4ms/step - loss: 8.2442 - RMSLE: 2.8450 - val_loss: 6.9234 - val_RMSLE: 2.6034
Epoch 5/100
188/188 [==============================] - 1s 5ms/step - loss: 8.0309 - RMSLE: 2.8066 - val_loss: 6.9293 - val_RMSLE: 2.6044
Epoch 6/100
188/188 [==============================] - 1s 5ms/step - loss: 7.8966 - RMSLE: 2.7854 - val_loss: 6.9488 - val_RMSLE: 2.6075


In [ ]:
model_cnnmlp.load_weights('cp3.hdf5')

pred_test = model_cnnmlp.predict(generator_test)

In [ ]:
d = np.array([test_set[["Id"]].to_numpy(), pred_test]).reshape((2,2500)).transpose()
res = pd.DataFrame(columns=['Id','Predicted'],index=None,data=d).set_index('Id')

In [ ]:
res.to_csv('submission.csv')

In [ ]:
! kaggle competitions submit -c ift6758-a20 -f submission.csv -m "without outliers and clustered color cleaned location"

100% 83.6k/83.6k [00:00<00:00, 198kB/s]
Successfully submitted to Social Media Prediction

In [ ]:
# res = np.array([test_set[["Id"]].to_numpy(), pred_test]).reshape((2,2500)).transpose()
# np.savetxt("foo.csv", res, delimiter=",", fmt="%s")